In [1]:
# Download TorchVision repo to use some files from
# references/detection
!git clone https://github.com/pytorch/vision.git
!cd vision
!git checkout v0.8.2

!cp ./vision/references/detection/utils.py ./
!cp ./vision/references/detection/transforms.py ./
!cp ./vision/references/detection/coco_eval.py ./
!cp ./vision/references/detection/engine.py ./
!cp ./vision/references/detection/coco_utils.py ./


Cloning into 'vision'...
remote: Enumerating objects: 590866, done.
remote: Counting objects: 100% (1239/1239), done.
remote: Compressing objects: 100% (932/932), done.
Receiving objects: 100% (590866/590866), 1.12 GiB | 4.53 MiB/s, done.
remote: Total 590866 (delta 1000), reused 351 (delta 301), pack-reused 589627 (from 5)
Resolving deltas: 100% (552022/552022), done.
error: pathspec 'v0.8.2' did not match any file(s) known to git


In [2]:
!pip install cython
# Install pycocotools, the version by default in Colab
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
!pip install -U albumentations
!pip install -U opencv-python

#Copy and unify the train and validation datasets into one folder for images and another for labels
!mkdir ./train
!cp -a /kaggle/input/martianlunar-crater-detection-dataset/craters/train/images/. ./train/images/
!cp -a /kaggle/input/martianlunar-crater-detection-dataset/craters/valid/images/. ./train/images/
!cp -a /kaggle/input/martianlunar-crater-detection-dataset/craters/train/labels/. ./train/labels/
!cp -a /kaggle/input/martianlunar-crater-detection-dataset/craters/valid/labels/. ./train/labels/

  Obtaining dependency information for cython from https://files.pythonhosted.org/packages/43/39/bdbec9142bc46605b54d674bf158a78b191c2b75be527c6dcf3e6dfe90b8/Cython-3.0.11-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 1.2 MB/s eta 0:00:00a 0:00:010m
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tables 3.8.0 requires blosc2~=2.0.0, which is not installed.
  Cloning https://github.com/cocodataset/cocoapi.git to /private/var/folders/n5/g9jzs83d0w34fq419d2bzwdh0000gn/T/pip-req-build-w7iffqka
  Running command git clone --filter=blob:none --quiet https://github.com/cocodataset/cocoapi.git /private/var/folders/n5/g9jzs83d0w34fq419d2bzwdh0000gn/T/pip-req-build-w7iffqka
  Resolved https://github.com/cocodataset/cocoapi.git to commit 8c9bcc3cf640524c4c20a9c40e89cb6a2f2fa0e9
  Preparing metadata (setup.py) ... done
  Created wheel

In [3]:

import os
import numpy as np
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from engine import train_one_epoch, evaluate
import utils
import transforms as T
import albumentations as A
import cv2
import time
from albumentations.pytorch.transforms import ToTensorV2
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from  sklearn.model_selection import KFold
import random

**Dataset class setup**

In [5]:
class CraterDataset(object):
    def __init__(self, root, transforms):
        self.root = root
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = list(sorted(os.listdir(os.path.join(self.root, "images"))))
        self.annots = list(sorted(os.listdir(os.path.join(self.root, "labels"))))
        self.classes = ['Background','Crater']

In [6]:
    # Converts boundry box formats, this version assumes single class only!
    def convert_box_cord(self,bboxs, format_from, format_to, img_shape):
        if format_from == 'normxywh':
            if format_to == 'xyminmax':
                xw = bboxs[:, (1, 3)] * img_shape[1]
                yh = bboxs[:, (2, 4)] * img_shape[0]
                xmin = xw[:, 0] - xw[:, 1] / 2
                xmax = xw[:, 0] + xw[:, 1] / 2
                ymin = yh[:, 0] - yh[:, 1] / 2
                ymax = yh[:, 0] + yh[:, 1] / 2
                coords_converted = np.column_stack((xmin, ymin, xmax, ymax))

        return coords_converted


In [7]:
    def __getitem__(self, idx):
        # load images and boxes
        img_path = os.path.join(self.root, "images", self.imgs[idx])
        annot_path = os.path.join(self.root, "labels", self.annots[idx])
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)
        img= img/255.0

    

In [8]:
       # retrieve bbox list and format to required type,
        # if annotation file is empty, fill dummy box with label 0
        if os.path.getsize(annot_path) != 0:
            bboxs = np.loadtxt(annot_path, ndmin=2)
            bboxs = self.convert_box_cord(bboxs, 'normxywh', 'xyminmax', img.shape)
            num_objs = len(bboxs)
            bboxs = torch.as_tensor(bboxs, dtype=torch.float32)
            # there is only one class
            labels = torch.ones((num_objs,), dtype=torch.int64)
            # suppose all instances are not crowd
            iscrowd = torch.zeros((num_objs,), dtype=torch.int64)
        else:
            bboxs = torch.as_tensor([[0, 0, 640, 640]], dtype=torch.float32)
            labels = torch.zeros((1,), dtype=torch.int64)
            iscrowd = torch.zeros((1,), dtype=torch.int64)

        area = (bboxs[:, 3] - bboxs[:, 1]) * (bboxs[:, 2] - bboxs[:, 0])
        image_id = torch.tensor([idx])

IndentationError: unexpected indent (1281466335.py, line 3)

In [7]:
class CraterDataset(object):
    def __init__(self, root, transforms):
        self.root = root
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = list(sorted(os.listdir(os.path.join(self.root, "images"))))
        self.annots = list(sorted(os.listdir(os.path.join(self.root, "labels"))))
        self.classes = ['Background','Crater']
        
    # Converts boundry box formats, this version assumes single class only!
    def convert_box_cord(self,bboxs, format_from, format_to, img_shape):
        if format_from == 'normxywh':
            if format_to == 'xyminmax':
                xw = bboxs[:, (1, 3)] * img_shape[1]
                yh = bboxs[:, (2, 4)] * img_shape[0]
                xmin = xw[:, 0] - xw[:, 1] / 2
                xmax = xw[:, 0] + xw[:, 1] / 2
                ymin = yh[:, 0] - yh[:, 1] / 2
                ymax = yh[:, 0] + yh[:, 1] / 2
                coords_converted = np.column_stack((xmin, ymin, xmax, ymax))

        return coords_converted

    def __getitem__(self, idx):
        # load images and boxes
        img_path = os.path.join(self.root, "images", self.imgs[idx])
        annot_path = os.path.join(self.root, "labels", self.annots[idx])
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)
        img= img/255.0

        # retrieve bbox list and format to required type,
        # if annotation file is empty, fill dummy box with label 0
        if os.path.getsize(annot_path) != 0:
            bboxs = np.loadtxt(annot_path, ndmin=2)
            bboxs = self.convert_box_cord(bboxs, 'normxywh', 'xyminmax', img.shape)
            num_objs = len(bboxs)
            bboxs = torch.as_tensor(bboxs, dtype=torch.float32)
            # there is only one class
            labels = torch.ones((num_objs,), dtype=torch.int64)
            # suppose all instances are not crowd
            iscrowd = torch.zeros((num_objs,), dtype=torch.int64)
        else:
            bboxs = torch.as_tensor([[0, 0, 640, 640]], dtype=torch.float32)
            labels = torch.zeros((1,), dtype=torch.int64)
            iscrowd = torch.zeros((1,), dtype=torch.int64)

        area = (bboxs[:, 3] - bboxs[:, 1]) * (bboxs[:, 2] - bboxs[:, 0])
        image_id = torch.tensor([idx])

        target = {}
        target["boxes"] = bboxs
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            sample = self.transforms(image=img,
                                     bboxes=target['boxes'],
                                     labels=labels)
        img = sample['image']
        target['boxes'] = torch.tensor(sample['bboxes'])
        target['labels'] = torch.tensor(sample['labels'])
        if target['boxes'].ndim == 1:
            target['boxes'] = torch.as_tensor([[0, 0, 640, 640]], dtype=torch.float32)
            target['labels'] = torch.zeros((1,), dtype=torch.int64)
        return img, target

    def __len__(self):
        return len(self.imgs)

Model Configuration

In [6]:
def get_model_bbox(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    
    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    return model

In [4]:
def get_transform(train):
    if train:
        return A.Compose([
            # A.Flip(p=0.5),
            # A.RandomResizedCrop(height=640,width=640,p=0.4),
            # # A.Perspective(p=0.4),
            # A.Rotate(p=0.5),
            # # A.Transpose(p=0.3),
            ToTensorV2(p=1.0)],
            bbox_params=A.BboxParams(format='pascal_voc',min_visibility=0.4, label_fields=['labels']))
    else:
        return A.Compose([ToTensorV2(p=1.0)],
                         bbox_params=A.BboxParams(format='pascal_voc', min_visibility=0.5, label_fields=['labels']))

In [5]:
def reset_weights(m):
  '''
    Try resetting model weights to avoid
    weight leakage.
  '''
  for layer in m.children():
    if hasattr(layer, 'reset_parameters'):
        print(f'Reset trainable parameters of layer = {layer}')
        layer.reset_parameters()

**Image visualization**
The following function displays an image recieved from the dataset and overlays the boundry boxes from the annotation file

In [2]:
# Function to visualize bounding boxes in the image
def plot_img_bbox(img, target):
    # plot the image and bboxes
    # Bounding boxes are defined as follows: x-min y-min width height
    fig, a = plt.subplots(1, 1)
    fig.set_size_inches(5, 5)
    a.imshow(img.permute((1,2,0)))
    for box in (target['boxes']):
        x, y, width, height = box[0], box[1], box[2] - box[0], box[3] - box[1]
        rect = patches.Rectangle((x, y),
                                 width, height,
                                 edgecolor='b',
                                 facecolor='none',
                                 clip_on=False)
        a.annotate('Crater', (x,y-20), color='blue', weight='bold',
                   fontsize=10, ha='left', va='top')

        # Draw the bounding box on top of the image
        a.add_patch(rect)
    plt.show()


In [8]:
dataset = CraterDataset('/Users/shashikumarezhil/Documents/Autonomous-Lander/Moon-landings/craters/train/images', get_transform(train=True))
# Prints an example of image with annotations
for i in random.sample(range(1, 100), 3):
    img, target = dataset[i]
    plot_img_bbox(img, target)

NameError: name 'A' is not defined

In [10]:
!pip install albumentations



In [11]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

def get_transform(train):
    if train:
        return A.Compose([
            # A.Flip(p=0.5),
            # A.RandomResizedCrop(height=640,width=640,p=0.4),
            # A.Perspective(p=0.4),
            # A.Rotate(p=0.5),
            # A.Transpose(p=0.3),
            ToTensorV2(p=1.0)],
            bbox_params=A.BboxParams(format='pascal_voc', min_visibility=0.4, label_fields=['labels']))
    else:
        return A.Compose([ToTensorV2(p=1.0)],
                         bbox_params=A.BboxParams(format='pascal_voc', min_visibility=0.5, label_fields=['labels']))

# Example usage
dataset = CraterDataset('/Users/shashikumarezhil/Documents/Autonomous-Lander/Moon-landings/craters/train/images', get_transform(train=True))

# Prints an example of image with annotations
for i in random.sample(range(1, 100), 3):
    img, target = dataset[i]
    plot_img_bbox(img, target)


/Users/shashikumarezhil/anaconda3/lib/python3.11/site-packages/albumentations/core/composition.py:250: UserWarning: Got processor for bboxes, but no transform to process it.
  self._set_keys()


NameError: name 'os' is not defined

In [13]:
import os

path = '/Users/shashikumarezhil/Documents/Autonomous-Lander/Moon-landings/craters/train/images'
files = os.listdir(path)
print(files)


['011_png.rf.8ac312b4898f0106d10b76952a55d237.jpg', '020_png.rf.ce87f4889d7441275135633392f98ed7.jpg', 'mars_crater--103-_jpg.rf.61597458910fb0eed55b415c9cbac3bf.jpg', 'mars_crater--27-_jpg.rf.158fc0c6a694bba2f0afa44fd76a7f5a.jpg', 'mars_crater--20-_jpg.rf.cb4c2603f4a62bbe96328c29cec6d68e.jpg', 'mars_crater--111-_jpg.rf.215f88d6bd7a6d86560754be694fc2f3.jpg', 'mars_crater--86-_jpg.rf.81699898067809e7560109fb72a56670.jpg', 'mars_crater--74-_jpg.rf.259a2b0955c98b2b468d63917ae13796.jpg', 'mars_crater--15-_jpg.rf.c669035a909b835be84fa4fc67bf3f40.jpg', 'mars_crater--62-_jpg.rf.18dd7779b62fb78d5f07060fec32079d.jpg', 'mars_crater--4-_jpg.rf.b31f215c27d917bdb7c7c545db2bc36b.jpg', 'mars_crater--88-_jpg.rf.2f3b851dec48a94bfccc6519b4a45859.jpg', '01_png.rf.4d2ebc5ed98ad1e69d667aadbce63d53.jpg', 'mars_crater--8-_jpg.rf.85af0fa193ad2d5b0087dc48ec7341c6.jpg', 'mars_crater--93-_jpg.rf.7426ac666cec74a8b0bc15d550b8699b.jpg', '022_png.rf.00ae4e655a2774bada1e254641482935.jpg', 'mars_crater--53-_jpg.rf.20a

In [16]:
import os
import random
import albumentations as A
from albumentations.pytorch import ToTensorV2

def get_transform(train):
    if train:
        return A.Compose([
            # A.Flip(p=0.5),
            # A.RandomResizedCrop(height=640,width=640,p=0.4),
            # A.Perspective(p=0.4),
            # A.Rotate(p=0.5),
            # A.Transpose(p=0.3),
            ToTensorV2(p=1.0)],
            bbox_params=A.BboxParams(format='pascal_voc', min_visibility=0.4, label_fields=['labels']))
    else:
        return A.Compose([ToTensorV2(p=1.0)],
                         bbox_params=A.BboxParams(format='pascal_voc', min_visibility=0.5, label_fields=['labels']))

# Example usage
# dataset = CraterDataset('/Users/shashikumarezhil/Documents/Autonomous-Lander/Moon-landings/craters/train/images', get_transform(train=True))

# Prints an example of image with annotations
for i in random.sample(range(1, 100), 3):
    img, target = dataset[i]
    plot_img_bbox(img, target)



NameError: name 'dataset' is not defined

In [17]:
import random
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Define your transformation function
def get_transform(train):
    if train:
        return A.Compose([
            # A.Flip(p=0.5),
            # A.RandomResizedCrop(height=640,width=640,p=0.4),
            # A.Perspective(p=0.4),
            # A.Rotate(p=0.5),
            # A.Transpose(p=0.3),
            ToTensorV2(p=1.0)],
            bbox_params=A.BboxParams(format='pascal_voc', min_visibility=0.4, label_fields=['labels']))
    else:
        return A.Compose([ToTensorV2(p=1.0)],
                         bbox_params=A.BboxParams(format='pascal_voc', min_visibility=0.5, label_fields=['labels']))

# Define or initialize your dataset variable
dataset = CraterDataset('/Users/shashikumarezhil/Documents/Autonomous-Lander/Moon-landings/craters/trainimport random
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Define your transformation function
def get_transform(train):
    if train:
        return A.Compose([
            # A.Flip(p=0.5),
            # A.RandomResizedCrop(height=640,width=640,p=0.4),
            # A.Perspective(p=0.4),
            # A.Rotate(p=0.5),
            # A.Transpose(p=0.3),
            ToTensorV2(p=1.0)],
            bbox_params=A.BboxParams(format='pascal_voc', min_visibility=0.4, label_fields=['labels']))
    else:
        return A.Compose([ToTensorV2(p=1.0)],
                         bbox_params=A.BboxParams(format='pascal_voc', min_visibility=0.5, label_fields=['labels']))

# Define or initialize your dataset variable
dataset = CraterDataset('/Users/shashikumarezhil/Documents/Autonomous-Lander/Moon-landings/craters/train/images', get_transform(train=True))

# Prints an example of image with annotations
for i in random.sample(range(1, 100), 3):
    img, target = dataset[i]
    plot_img_bbox(img, target)
', get_transform(train=True))

# Prints an example of image with annotations
for i in random.sample(range(1, 100), 3):
    img, target = dataset[i]
    plot_img_bbox(img, target)


FileNotFoundError: [Errno 2] No such file or directory: '/Users/shashikumarezhil/Documents/Autonomous-Lander/Moon-landings/craters/train/images/images'

In [18]:
from PIL import Image

class CraterDataset:
    def __init__(self, root, transform=None):
        self.root = root
        self.transform = transform
        self.imgs = sorted(os.listdir(os.path.join(self.root, "images")))  # Listing image files

    def __getitem__(self, idx):
        img_name = self.imgs[idx]
        img_path = os.path.join(self.root, "images", img_name)
        
        img = Image.open(img_path)
        target = {}  # Add annotations or any other necessary information
        
        # Apply transformations if any
        if self.transform:
            img = self.transform(image=img)["image"]
        
        return img, target

# Example usage
dataset = CraterDataset('/Users/shashikumarezhil/Documents/Autonomous-Lander/Moon-landings/craters/train', transform=get_transform(train=True))

# Check if the dataset is correctly loading images
img, target = dataset[0]  # Access the first image
img.show()  # Display the image


TypeError: image must be numpy array type

In [19]:
print(dataset[0]) 

TypeError: image must be numpy array type

In [20]:
dataset = CraterDataset('/Moon-landings/craters/train', get_transform(train=True))
# Prints an example of image with annotations
for i in random.sample(range(1, 100), 3):
    img, target = dataset[i]
    plot_img_bbox(img, target)

FileNotFoundError: [Errno 2] No such file or directory: '/Moon-landings/craters/train/images'

In [21]:
import random
from Moo import CraterDataset, plot_img_bbox  # Make sure to import necessary modules

# Initialize the dataset
dataset = CraterDataset('/Moon-landings/craters/train', get_transform(train=True))

# Prints examples of images with annotations
for i in random.sample(range(1, len(dataset)), 3):  # Ensure you are sampling within the dataset size
    img, target = dataset[i]
    plot_img_bbox(img, target)


ModuleNotFoundError: No module named 'your_dataset_module'